# Creating Customer Segments

## 1. Aim

In this project, a dataset containing data on various customers' annual spending amounts of diverse product categories will be analyzed. One goal of this project is to best describe the variation in the different types of customers that a wholesale distributor interacts with. Doing so would equip the distributor with insight into how to best structure their delivery service to meet the needs of each customer.

The dataset for this project can be found in https://archive.ics.uci.edu/ml/datasets/Wholesale+customers. For the purposes of this project, the features `'Channel'` and `'Region'` will be excluded in the analysis — with focus instead on the six product categories recorded for customers.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the wholesale customers dataset
try:
    data = pd.read_csv("customers.csv")
    data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    print("Wholesale customers dataset has {} samples with {} features each.".format(*data.shape))
except:
    print("Dataset could not be loaded. Is the dataset missing?")

## 2. Data Exploration
A statistical description of the dataset will be observed below. Note that the dataset is composed of six important product categories: **'Fresh'**, **'Milk'**, **'Grocery'**, **'Frozen'**, **'Detergents_Paper'**, and **'Delicatessen'**.

In [ ]:
# Display a description of the dataset
display(data.describe())

### Selecting Samples
To get a better understanding of the customers and how their data will transform through the analysis, 3 sample data points will be explored and track through the course of the project.

In [ ]:
indices = [1,70,400]
samples = pd.DataFrame(data.loc[indices], columns = data.keys()).reset_index(drop = True)
print("Chosen samples of wholesale customers dataset:")
display(samples)

How do the samples compare? What kind of establishments might they be? 
Considering the total purchase cost of each product category and the statistical description of the dataset, with the following mean values:

The first client could be a wholesale retailer because there is a lower than average portion of fresh food (both with respect to the rest of the categories and also a lower than average amount with respect to the rest of the clients) and a higher portion of the milk & grocery, while there are a similar proportion of Detergents_Paper and Delicatessen. It also has a rather high volume in most categories, 
The second client could be a fast food chain that buys in bulk fresh food (I would imagine some veggies to add to burgers or tacos), a low amount of milk and detergents (they use disposable utensils so the detergent needed is only for the upkeep of the kitchen) and a high amount of frozen food (could be burgers, nuggets, fries, etc)
The last one could be a diner with a smaller size that the 2nd client, because it does have a high percentage of fresh food, grocery and frozen food; but milk is rather low, and detergent is low but in relative terms it is not that low, so they might not be using disposables.

### Feature Relevance
One interesting thought to consider is if one (or more) of the six product categories is actually relevant for understanding customer purchasing. That is to say, is it possible to determine whether customers purchasing some amount of one category of products will necessarily purchase some proportional amount of another category of products? We can make this determination quite easily by training a supervised regression learner on a subset of the data with one feature removed, and then score how well that model can predict the removed feature.

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Split the data into training and testing sets using the given feature ('Delicatessen') as the target
new_data = data.drop('Delicatessen',axis=1)
X_train, X_test, y_train, y_test = train_test_split(new_data,data['Delicatessen'],test_size=0.25,random_state=0)
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)

score = regressor.score(X_test,y_test)
score

The reported score for coefficient of determination was -11.66. 
I consider that this is key to identifying the costumers. The negative Rsquared shows that the Delicatessen feature is hard to predict from the other features, thus it contains new information that can help us identify different patterns in customers' spending habits. We should note that when taking out the seeds for random states this result changes (though it doesn't change enough so as to have a high score).

Also, logically, the amount and proportion of Delicatessen supply will be different if the costumer is a fancy restaurant (that sells these items and thus might have a higher demand), or if it is a fast food chain or a wholesale retailer (which doesn't typically offer these items). Therefore, there probably is some correlation between the type of business of the client and the variable Delicatessen.

### Visualize Feature Distributions
To get a better understanding of the dataset, we can construct a scatter matrix of each of the six product features present in the data. If a feature is not relevant for identifying a specific customer, the scatter matrix might show a correlation between that feature and another feature in the data.

In [ ]:
pd.scatter_matrix(data, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(data.corr())
plt.title("Pearson's correlation among features",fontsize=16)
plt.show()

The distribution is not particularly normal, the data presents a highly skewed distribution to the right in all cases, due to the long right tail formed by outliers. This can make it tricky to decide whether a point should be treated as an outlier or not, for example if we try to use the 2 or 3 standard deviations as reference, or the whiskers of a boxplot, or even an adjusted boxplot (such as Vandervieren's) - but we can try transforming the variables with the logarithm.

Some of the features do exhibit certain correlation with the rest, for example detergents_paper is the feature that more correlation displays with the rest, and particularly it is highly correlated with Grocery.

Looking at the scatterplots we can confirm that Delicatessen is not particularly correlated to the rest of the features, but to enphasize it we can see the heatmap of Pearson's correlation, where we see that fresh, frozen and delicatessen are the categories that tend to be more uncorrelated.

## 3. Data Preprocessing
In this section, we will preprocess the data to create a better representation of customers by performing a scaling on the data and detecting (and optionally removing) outliers.

### Feature Scaling
Since data is not normally distributed (especially if there is a large skew), it is most often appropriate (http://econbrowser.com/archives/2014/02/use-of-logarithms-in-economics) to apply a non-linear scaling — particularly for financial data. One way to achieve this scaling is by using a [Box-Cox test](http://scipy.github.io/devdocs/generated/scipy.stats.boxcox.html), which calculates the best power transformation of the data that reduces skewness. A simpler approach which can work in most cases would be applying the natural logarithm.

In [ ]:
log_data = np.log(data)
log_samples = np.log(samples)
pd.scatter_matrix(log_data, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [ ]:
display(log_samples)

### Outlier Detection
Detecting outliers in the data is extremely important in the data preprocessing step of any analysis. The presence of outliers can often skew results which take into consideration these data points. There are many "rules of thumb" for what constitutes an outlier in a dataset. Here, we will use [Tukey's Method for identfying outliers](https://www.kdnuggets.com/2017/01/3-methods-deal-outliers.html): An *outlier step* is calculated as 1.5 times the interquartile range (IQR). 

A data point with a feature that is beyond an outlier step outside of the IQR will be considered abnormal. For those data points that are outliers in at least 2 features, they will definitely not be representative of a costumer consumption pattern.

It is important to drop these outliers since many clustering algorithms (like kmeans) depend on the data being neatly separated in clusters per class to work well, so outliers can greatly impact our decision boundries.

In [ ]:
outliers_index = []#
for feature in log_data.keys():
    Q1 = np.percentile(log_data[feature],25)
    Q3 = np.percentile(log_data[feature],75)
    interquartile_range = Q3-Q1
    # calculate an outlier step (1.5 times the interquartile range)
    step = 1.5*interquartile_range
    print("Data points considered outliers for the feature '{}':".format(feature))
    outliers = log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))]
    display(outliers)
    outliers_index.extend(list(outliers.index))#


outliers = [65,66,75,128,154] #del!!!
outliers = np.unique(outliers_index,return_counts=True)#
outliers = np.where(outliers>1)#
good_data = log_data.drop(log_data.index[outliers]).reset_index(drop = True)

## 4. Feature Transformation
In this section, principal component analysis (PCA) will be used to draw conclusions about the underlying structure of the wholesale customer data. Since using PCA on a dataset calculates the dimensions that best maximize variance, we will find which compound combinations of features best describe customers.

### PCA

Now that the data has been scaled to a more normal distribution and has had any necessary outliers removed, we can now apply PCA to the `good_data` to discover which dimensions about the data best maximize the variance of features involved. In addition to finding these dimensions, PCA will also report the *explained variance ratio* of each dimension — how much variance within the data is explained by that dimension alone. Note that a component (dimension) from PCA can be considered a new "feature" of the space, however it is a composition of the original features present in the data.

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA by fitting the good data with the same number of dimensions as features
pca = PCA(n_components=len(good_data.columns)).fit(good_data)
print('explained variance when fitting good data:',pca.explained_variance_ratio_)

# Transform log_samples using the PCA fit above
pca_samples = pca.transform(log_samples)
print('explained variance when transforming log samples will remain the same because it is still fitted with good_data:',pca.explained_variance_ratio_)

# Generate PCA results plot
pca_results = vs.pca_results(good_data, pca)

70.681723% of the variance of the data is explained by the first and second principal components, and this percentage increases to 93.109011% when taking the first 4 principal components.

Let's touch upon which features are well represented by each dimension(both in terms of positive and negative variance explained). For this, please note that a positive increase in a specific dimension corresponds with an *increase* of the *positive-weighted* features and a *decrease* of the *negative-weighted* features. The rate of increase or decrease is based on the individual feature weights.

- The first component has a strong correlation with the demand of detergents and paper and in less measure with grocery and milk. Delicatessen, fresh and frozen show a lower weight, which means that those dimensions will impact less in this 1st component. This could help identify wholesale retailers or places oriented to customers where there is a lot of cleaning and a small amount of feeding, such as an hotel or hospital.

- The second component is all negative, which is not a problem since the negative sign is not the same as having a negative correlation, it is simply the direction of the eigenvectors: a positive or negative PC it just means that you are projecting on an eigenvector that is pointing in one direction or 180∘ away in the other direction, as can be seen in the following thread: https://stats.stackexchange.com/questions/88880/does-the-sign-of-scores-or-of-loadings-in-pca-or-fa-have-a-meaning-may-i-revers. This component assigns special importance to fresh, frozen and delicatessen, which might be helpful to identify food places such as restaurants.

- Similarly, the 3rd one also shows a big percentage of the variance to be explained by fresh delicatessen and frozen, but unlike the previous one this one takes into consideration the features frozen and delicatessen in the opposite way, as the eigenvector is proyected 180° away from fresh. This could add information to the second component by helping distinguish between a sophisticated/exotic restaurant from a place that sells food from fresh items, as could be a salad place.

- The fourth component shows a big percentage explained by frozen and delicatessen, it is separating the effects by proyectim them 180° apart, so it could be considered to be adding to the 2nd and 3rd component to help distinguish between places that are an exotic restaurant or a fast food place.


In [ ]:
# Display sample log-data after having a PCA transformation applied
display(pd.DataFrame(np.round(pca_samples, 4), columns = pca_results.index.values))

### Dimensionality Reduction
When using principal component analysis, one of the main goals is to reduce the dimensionality of the data — in effect, reducing the complexity of the problem. Dimensionality reduction comes at a cost: Fewer dimensions used implies less of the total variance in the data is being explained. Because of this, the *cumulative explained variance ratio* is extremely important for knowing how many dimensions are necessary for the problem. Additionally, if a signifiant amount of variance is explained by only two or three dimensions, the reduced data can be visualized afterwards.

In [ ]:
# Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components=2).fit(good_data)
reduced_data = pca.transform(good_data)
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

In [ ]:
# Display sample log-data after applying PCA transformation in two dimensions
pca_samples = pca.transform(log_samples)
display(pd.DataFrame(np.round(pca_samples, 4), columns = ['Dimension 1', 'Dimension 2']))
print('The values for the first two dimensions remains unchanged when compared \
      to a PCA transformation in six dimensions.')


## 5. Clustering

In this section, either a K-Means clustering algorithm or a Gaussian Mixture Model clustering algorithm will be chosen to identify the various customer segments hidden in the data. Afterwards, specific data points from the clusters will be recovered, to understand their significance by transforming them back into their original dimension and scale. 

Some advantages to using a K-Means are computational time, reliability (if the model is appropiate for the dataset),though we should keep in mind that this algorithm makes the assumption that the clusters are of similar size, that the features in a cluster have a similar and small variance and that data that is most suitable for this algorithm is clustered in structures resembling small spheres.
Some of the advantages of using GMM are that we gain flexibility since it doesn't assume that the features in a cluster have the same variance, this allows us to identify more complex clusters as we ar able to incorporate the covariance between the pints in our model. Also, instead of having the most probable cluster, we get the probability of belonging to every cluster. 
Given that GMM is more flexible than kmeans and that we dont have a lot of data (so speed will not have a great impact), I will use GMM.

### Creating Clusters
When the number of clusters is not known *a priori*, there is no guarantee that a given number of clusters best segments the data, since it is unclear what structure exists in the data — if any. However, we can usually quantify the "goodness" of a clustering by calculating each data point's *silhouette coefficient*. The [silhouette coefficient](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). Calculating the *mean* silhouette coefficient provides for a simple scoring method of a given clustering.

In [ ]:
from sklearn.mixture import GMM
from sklearn.metrics import silhouette_score

clusterer = GMM(n_components=2).fit(reduced_data)
preds = clusterer.predict(reduced_data)
centers = clusterer.means_
sample_preds = clusterer.predict(pca_samples)

score = silhouette_score(reduced_data,preds)
score

**Silhouette score:**
2 clusters: 0.4118
3 clusters:0.3736
4 clusters:0.3328
5 clusters: 0.2809
The silhouette score suggets that 2 clusters is the best option

### Cluster Visualization

In [ ]:
vs.cluster_results(reduced_data, preds, centers, pca_samples)

### Data Recovery
Each cluster present in the visualization above has a central point. These centers (or means) are not specifically data points from the data, but rather the *averages* of all the data points predicted in the respective clusters. For the problem of creating customer segments, a cluster's center point corresponds to *the average customer of that segment*. Since the data is currently reduced in dimension and scaled by a logarithm, we can recover the representative customer spending from these data points by applying the inverse transformations.

In [ ]:
# Recover
log_centers = pca.inverse_transform(centers)
true_centers = np.exp(log_centers)

# Display the true centers
segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(true_centers), columns = data.keys())
true_centers.index = segments
display(true_centers)

Looking at the means in the original data (in the cell below), we notice that both clusters 1 and 2 have below average demand of fresh, frozen and delicatessen. Regarding the difference among these clusters, we wee that cluster 1 has an above average demand of Detergents_Paper, grocery and milk. Combined with the fact that they have also a low demand of delicatessen and fresh, I would venture that this kind of costumer is probably a wholesale retailer.
As for the other cluster, we see an extremely low demand of Detergents_Paper, a nearly average demand of frozen, and a below average but still important amount of fresh food. In general terms we see that this is a smaller costumer, that has an overall larger than average

In [ ]:
data.describe().loc['mean']

As for the other cluster, we see an extremely low demand of Detergents_Paper, a nearly average demand of frozen, and a below average but still important amount of fresh food. In general terms we see that this is a smaller costumer, so we will be better off judging it considering it in percentaje. Bellow we can see the proportion that an average costumer demands of each category, and the proportion that the costumers in the 2nd cluster demand on average from each category.

In [ ]:
print('proportion that an average costumer demands of each category')
display(data.describe().loc['mean']/np.sum(data.describe().loc['mean']))
print('proportion that the costumers in the 2nd cluster demand on average from each category')
true_centers.loc['Segment 1']/np.sum(true_centers.loc['Segment 1'])

From the above proportions we can see that has an overall larger than average demand of fresh and frozen, and the amount of milk and grocery is quite large despite being below average. It would seem that the 2nd cluster could represent food vendors such as restaurants, dinners, fast food, etc. Apart from food vendors, this could also include other businesses that involve feeding costumers, such as hospitals or hotels.

Below we'll try to make sense of the analysis looking at the sample points.
Are the predictions for each of them consistent with our original idea?

In [ ]:
# Display the predictions
for i, pred in enumerate(sample_preds):
    print("Sample point", i, "predicted to be in Cluster", pred)

In [ ]:
print('recall the values in the samples:')
display(samples)

In [ ]:
print('the centers of the clusters are:')
display(true_centers)

- The first sample point seems to be a wholesale retailer. When we look at the values in the 1st sample compared to the cluster centers we see that despite having the dimension fresh nearer to the 1st cluster (segment 0), we see that milk, grocery, detergents and delicatessen are noticeably nearer to the 2nd cluster (segment 1), so it makes sense that we had this prediction.

- The second sample seems to be a restaurant, we see that despite having delicatessen much nearer to the 2nd cluster, all the other variables are much closer to the 1st cluster (food vendors), so the initial intuition of this sample being a restaurant was correctly predicted by the model as well.

- The third sample seems was expected to be a diner, and this was also predicted by the model (which found it to belong to the 1st cluster). Wnen comparing the cluster centers to the 3rd sample values, we see that Milk, Grocery, Frozen and Detergents_Paper are very near to the 1st cluster, so the prediction that this is a food retailer makes sense, and it reafirms the initial thoughts of this being a diner.

## 6. Conclusion

Additional structure is derived from originally unlabeled data when using clustering techniques. Since each customer has a ***customer segment*** it best identifies with (depending on the clustering algorithm applied), we can consider *'customer segment'* as an **engineered feature** for the data. Assume the wholesale distributor recently acquired ten new customers and each provided estimates for anticipated annual spending of each product category. Knowing these estimates, the wholesale distributor wants to classify each new customer to a ***customer segment*** to determine the most appropriate delivery service.  
* How can the wholesale distributor label the new customers using only their estimated product spending and the **customer segment** data?

We could use a supervised learner such as a decision tree to estimate the target variable, which will be  'customer segment', our recently engineered feature. We would train it with the current data and predict on the new costumer estimated product spending.

### Visualizing Underlying Distributions

At the beginning of this project, it was discussed that the `'Channel'` and `'Region'` features would be excluded from the dataset so that the customer product categories were emphasized in the analysis. By reintroducing the `'Channel'` feature to the dataset, an interesting structure emerges when considering the same PCA dimensionality reduction applied earlier to the original dataset.

Below we see how each data point is labeled either `'HoReCa'` (Hotel/Restaurant/Cafe) or `'Retail'` the reduced space. In addition, the sample points are circled in the plot, which will identify their labeling.

In [ ]:
# Display the clustering results based on 'Channel' data
vs.channel_results(reduced_data, outliers, pca_samples)

Comparing the clustering algorithm used and the results adding the channel information, we can see that there is a big resemblance. There are 2 clusters in both cases and the costumer segments are similarly separated. The costumer segments expected to be Hotel/estaurant/Cafe were in fact correct, and the same for retail.There is of course some noise that we weren't able to capture with the clustering model, but that is expected. Overall, the classification is very consistent with the one found by the clustering algorithm.